In [1]:
import pandas as pd
import numpy as np
from __future__ import division
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time

In [34]:
def calculateNewTotal(row,scores):
    if row['tripped']:
        tripped_df = pd.DataFrame(row['tripped'])
        tripped_df.columns = ['rule_id']
        tripped_score = pd.merge(tripped_df,scores,on='rule_id',how='inner')
        return tripped_score['score'].sum() + row['Score']
    
    else:
        return row['Score']

def comparescore(x,scorecol,treshold,compare):
    if isinstance(treshold, int):
        if compare == '<':
            if x[scorecol] < treshold:
                return 1
            else:
                return 0
        elif compare == '>':
            if x[scorecol] >= treshold:
                return 1
            else:
                return 0
    
    elif isinstance(treshold,tuple):
        t1 = min(treshold)
        t2 = max(treshold)
        
        if compare == '<':
            if x[scorecol] < t1:
                return 2
            elif x[scorecol] >= t1 and x[scorecol] < t2:
                return 1
            else:
                return 0
        elif compare == '>':
            if x[scorecol] >= t2:
                return 2
            elif x[scorecol] < t2 and x[scorecol] >= t1:
                return 1
            else:
                return 0
        
        
def getPredictionCounts(data,comp,treshold, scorecol):
    data['prediction_correct'] = data.apply(lambda x: comparescore(x,scorecol,treshold,comp),axis = 1)
    correct = data['prediction_correct'].value_counts()
    if isinstance(treshold, int):
        if 0 not in correct.index.values:
            correct[0] = 0
        elif 1 not in correct.index.values:
            correct[1] = 0
        
    elif isinstance(treshold,tuple):   
        if 0 not in correct.index.values:
            correct[0] = 0
        elif 1 not in correct.index.values:
            correct[1] = 0
        elif 2 not in correct.index.values:
            correct[2] = 0

    return correct

In [72]:
def simulate(fraud,normal,scores,treshold=5000):
    fraud['total'] = fraud.apply(lambda x: calculateNewTotal(x,scores),axis=1)
    normal['total'] = normal.apply(lambda x: calculateNewTotal(x,scores),axis=1)
    
    fraud_correct_before = getPredictionCounts(data = fraud, comp = '>', treshold = treshold, scorecol = 'Score')
    normal_correct_before = getPredictionCounts(data = normal, comp = '<', treshold = treshold, scorecol = 'Score')
    
    fraud_correct_after = getPredictionCounts(data = fraud, comp = '>', treshold = treshold, scorecol = 'total')
    normal_correct_after = getPredictionCounts(data = normal, comp = '<', treshold = treshold, scorecol = 'total')
    
    before_cm = createCM(fraud_correct_before,normal_correct_before)
    after_cm = createCM(fraud_correct_after,normal_correct_after)
    
    return pd.DataFrame(before_cm), pd.DataFrame(after_cm)

In [22]:
def calcFscore(cm):
    return 2*cm[0][0]/(2*cm[0][0] + cm[1][0] + cm[0][1])

def calcTPR(cm):
    return cm[0][0]/(cm[0][0] + cm[0][1])

def calcFPR(cm):
    return cm[1][0]/(cm[1][0] + cm[1][1])

def calcAccuracy(cm):
    return (cm[0][0] + cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])

def createCM(fraud_predict,normal_predict):
    return [[fraud_predict[i] for i in range(len(fraud_predict.index.values))],
            [normal_predict[i] for i in range(len(normal_predict.index.values)-1,-1,-1)]]   

In [5]:
def getRuleList(df):
    
    res = pd.DataFrame()
    res['tripped'] = ''
    res['tripped'] = res['tripped'].astype('object')
    scores = df.pop('Score')
    for index,row in df.iterrows():
        
        row_tx = row.pop('トランザクションID')
        row_t = row.T
        row_t = row_t[row_t != 0]
        res.at[index,'tripped'] = list(row_t.index.values)
        res.at[index,'トランザクションID'] = row_tx
    
    res.insert(2,'Score',scores)
    
    return res

In [26]:
scores = pd.read_csv('score_file.csv')

In [7]:
%%time
fraud_df = pd.read_csv('data/CB_sim2_new_rule_flgs.csv',encoding='SHIFT-JIS')
fraud_df = getRuleList(fraud_df)

Wall time: 3.06 s


In [9]:
%%time
normal_df = pd.read_csv('data/NotCB_sim2_new_rule_flgs.csv',encoding='SHIFT-JIS')
normal_df = normal_df.sample(frac = 0.05)
normal_df = getRuleList(normal_df)

Wall time: 3min 26s


In [105]:
%%time
treshold = (4000,5000)
bef,aft = simulate(fraud_df,normal_df,scores,treshold)

Wall time: 1min 19s


In [106]:
def formatResults(cm):
    cm.loc[-1] = cm.apply(lambda x: x.sum(),axis=0)
    cm.index = cm.index + 1  # shifting index
    cm = cm.sort_index() 
    cm = cm.rename({0:'Total',1:'CB',2:'NOTCB'},axis='index')
    if len(cm.columns) == 2:
        cm.columns = ['Accept(-' + str(treshold) + ')','Reject(' + str(treshold) + '-)']
        
    elif len(cm.columns) == 3:
        cm.columns = ['Accept(-' + str(min(treshold)) + ')',
                      'Review(' + str(min(treshold)) + '-' + str(max(treshold)) + ')',
                      'Reject(' + str(max(treshold)) + '-)']
    
    return cm

def normalizeResults(cm):
    norm_cm = 100*cm.div(cm.sum(axis=1), axis=0)
    return norm_cm

def addTotal(cm):
    cm['Total'] = cm.apply(lambda x: x.sum(),axis = 1)


In [83]:
bef

0    1
0   1769    5
1  29385  281

In [37]:
aft

[[1335, 439], [29455, 211]]

In [107]:
bef = formatResults(bef)
aft = formatResults(aft)
bef_n = normalizeResults(bef)
aft_n = normalizeResults(aft)
addTotal(bef)
addTotal(aft)
addTotal(bef_n)
addTotal(aft_n)

In [95]:
bef_n

Accept             Reject  Total
Total  99.090330788804067  0.909669211195929  100.0
CB     99.718151071025929  0.281848928974070  100.0
NOTCB  99.052787703094452  0.947212296905548  100.0

In [122]:
writer = pd.ExcelWriter('data/pandas_simple.xlsx', engine='xlsxwriter')

bef.to_excel(writer, sheet_name='report',startrow = 3,startcol = 2)
bef_n.to_excel(writer, sheet_name='report',startrow = 9,startcol = 2)

aft.to_excel(writer, sheet_name='report',startrow = 3,startcol = 9)
aft_n.to_excel(writer, sheet_name='report',startrow = 9, startcol = 9)

workbook = writer.book
worksheet = writer.sheets['report']
worksheet.set_zoom(120)

cell_format1 = workbook.add_format()
cell_format1.set_font_name('Yu Gothic')

worksheet.conditional_format('C4:N13', {'type': 'no_errors','format': cell_format1})

# Close the Pandas Excel writer and output the Excel file.
writer.save()